# **Test sentiment analysis model building code.**

**Importing the required libraries**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pickle

**Connecting Google drive**

**Not necessary if run on a local machine instead of google colab.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Loading the training data**

**Check for the file path if run in local machine**

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/test/Sentiment2.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

**Text preprocessing, analysing training data, and creating tokennizer**

In [ ]:
#data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)
print(data[ data['sentiment'] == 'Neutral'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

4578
17054
6380


**Model Creation**

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 28, 128)           256000    
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 28, 128)          0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 196)               254800    
                                                                 
 dense_1 (Dense)             (None, 3)                 591       
                                                                 
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


**Spliting the data in train test split**

In [ ]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9384, 28) (9384, 3)
(4622, 28) (4622, 3)


**Training the model with training part**

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 3)

Epoch 1/7
Epoch 2/7
Epoch 3/7
Epoch 4/7
Epoch 5/7
Epoch 6/7
Epoch 7/7


**Validating the model accuracy and Score with test data**

In [ ]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

98/98 - 2s - loss: 0.8343 - accuracy: 0.6726 - 2s/epoch - 25ms/step
score: 0.83
acc: 0.67


**Classification wise accuracy checking**

In [ ]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 0)
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        elif np.argmax(Y_validate[x]) == 2:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    elif np.argmax(Y_validate[x]) == 2:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 45.52845528455284 %
neg_acc 82.77356446370531 %


**Saving the model in a file**


**<<Don't run this part unless you want to override existing model>>**

**Check for the file path if run in local machine**

In [ ]:
fname = '/content/drive/MyDrive/test/finalized_text_model.sav'
pickle.dump(model, open(fname, 'wb'))
tokenname = '/content/drive/MyDrive/test/text_model_token.sav'
pickle.dump(tokenizer, open(tokenname, 'wb'))

INFO:tensorflow:Assets written to: ram://74731b5e-4b3a-4981-a7dc-b8151de4eeb7/assets


**Not necessary**

In [ ]:
text = input()

txt =[]
txt.append(text)
#print(emoji.demojize(emo))
#print(txt)
txt = tokenizer.texts_to_sequences(txt)
#padding the text to have exactly the same shape as `embedding_2` input
txt = pad_sequences(txt, maxlen=28, dtype='int32', value=0)
#print(twt)
text_sentiment = model.predict(txt,batch_size=1)
print("\n\nTone of the text part is ", end = " :: ")
if(np.argmax(text_sentiment) == 0): print("negative"); text_sentiment = 'negative'
elif(np.argmax(text_sentiment) == 1): print("neutral"); text_sentiment = 'neutral'
elif (np.argmax(text_sentiment) == 2): print("positive"); text_sentiment = 'positive'

I got approved


Tone of the text part is  :: neutral
